In [1]:
import cv2
import torch
import numpy as np
from models.i3d.i3d import InceptionI3d
import lightning as L
from torch.nn import functional as F
from datamodule.wlasl import WLASLDataModule

Matplotlib created a temporary cache directory at /tmp/matplotlib-sqofkf30 because the default path (/teamspace/studios/this_studio/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def video_to_tensor(path, frames_count):
    vid = cv2.VideoCapture(video_path)
    vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frames = []

    while True:
        success, frame = vid.read()

        if not success:
            break
        
        if len(frames) >= frames_count:
            break

        w, h, c = frame.shape
        if w < 226 or h < 226:
            d = 226.0 - min(w, h)
            sc = 1 + d / min(w, h)
            frame = cv2.resize(frame, dsize=(0, 0), fx=sc, fy=sc)

        if w > 256 or h > 256:
            new_h = math.ceil(h * (256 / h))
            new_w = math.ceil(w * (256 / w))
            frame = cv2.resize(frame, dsize=(new_w, new_h))

        frame = (frame / 255.0) * 2 - 1

        frames.append(frame)

    np_frames = np.asarray(frames, dtype=np.float32)

    return torch.from_numpy(np_frames.transpose(3, 0, 1, 2))

In [3]:
video_path = "WLASL_2/WLASL2000/00295.mp4"
count = 64

frames = video_to_tensor(video_path, count)
frames= frames.unsqueeze(0)

In [4]:
checkpoint_path = "checkpoints/last.ckpt"
model = InceptionI3d.load_from_checkpoint(checkpoint_path=checkpoint_path, num_classes=2000)
model.eval()

InceptionI3d(
  (avg_pool): AvgPool3d(kernel_size=[2, 7, 7], stride=(1, 1, 1), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (logits): Conv3dBlock(
    (conv3d): Conv3d(1024, 2000, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
  (Conv3d_1a_7x7): Conv3dBlock(
    (conv3d): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), bias=False)
    (bn): BatchNorm3d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (MaxPool3d_2a_3x3): MaxPool3dDynamicPadding(kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (Conv3d_2b_1x1): Conv3dBlock(
    (conv3d): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (Conv3d_2c_3x3): Conv3dBlock(
    (conv3d): Conv3d(64, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (MaxPoo

In [5]:
data_dir = "WLASL_2/WLASL2000"
annotations_path = "WLASL/code/preprocess/nslt_2000.json"

In [6]:
trainer = L.Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
dm = WLASLDataModule(
        data_dir=data_dir,
        annotations=annotations_path,
    )

In [8]:
result = trainer.predict(model, datamodule=dm)

You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [9]:
result

[(tensor(481), tensor(481), ('65097',)),
 (tensor(1164), tensor(1164), ('35544',)),
 (tensor(1164), tensor(1164), ('35548',)),
 (tensor(208), tensor(208), ('24029',)),
 (tensor(1929), tensor(1929), ('51829',)),
 (tensor(969), tensor(969), ('48027',)),
 (tensor(1172), tensor(1172), ('38842',)),
 (tensor(1052), tensor(1052), ('05798',)),
 (tensor(1164), tensor(1164), ('35545',)),
 (tensor(998), tensor(998), ('55292',)),
 (tensor(998), tensor(998), ('55293',)),
 (tensor(352), tensor(352), ('02109',)),
 (tensor(998), tensor(998), ('55296',)),
 (tensor(998), tensor(998), ('55295',)),
 (tensor(352), tensor(352), ('02103',)),
 (tensor(187), tensor(187), ('07460',)),
 (tensor(187), tensor(187), ('07461',)),
 (tensor(352), tensor(352), ('02106',)),
 (tensor(352), tensor(352), ('02104',)),
 (tensor(676), tensor(676), ('16073',)),
 (tensor(239), tensor(239), ('48898',)),
 (tensor(239), tensor(239), ('48899',)),
 (tensor(821), tensor(821), ('06039',)),
 (tensor(534), tensor(534), ('22269',)),
 (te